# 1. Fasttrack setup

Assuming that the yml-files are edited to correspond to your actual setup, then the following steps will create the SimpliVity-Connector together with Prometheus and Grafana environment. 

If you want to take a look into the details and go step by step using this Jupyter notebook, then jump to chapter 2. 

In [ ]:
kubectl apply -f namespace.yml             # create the namespace
kubectl apply -f svtdemo.yml               # apply the SimpliVity connector configmap (in the following example the name svtdemo was used)
kubectl apply -f svtconnector.yml          # create the SimpliVity connector pod
kubectl apply -f prometheus.configmap.yml  # create the Prometheus configmap
kubectl apply -f prometheus.pv-claim.yml   # create the persistent volume for the Prometheus database
kubectl apply -f prometheus.yml            # create the Prometheus pod
kubectl apply -f grafana.pv-claim.yml      # create the persistent volume for the Grafana database
kubectl apply -f grafana.yml               # create the grafana pod

# 2. Detailed Setup 

## 2.1 Create the namespace
Create a namespace for your deployment. The example below creates the namespace svtprometheus.

In [ ]:
cat << 'EOF' | kubectl apply -f -
apiVersion: v1
kind: Namespace
metadata:
  name: svtprometheus
EOF

## 2.2 Create the SimpliVity - Prometheus connector POD

### 2.2.1 Create the ConfigMap

The connection and monitoring parameters are transferred to the HPE SimpliVity Prometheus connector as a Kubernetes ConfigMap. 

The ConfigMap can be created using th Python script: CreateConfigMap.py. 

Requirements:
    - System with Python 3 and the following Python packages installed: 
      - Fernet
      - getpass
      - etree

Run the script with the following command: 

#### python3 CreateConfigMap.py

The script will ask for the following information:
    - username               vCenter username (a user with readonly access rights is sufficient)
    - password               vCenter password
    - OVC/MVA IP address     IP address that the connector uses to connect to the federation
    - name                   name of the yml-file (<name>.yml) and the configmap: <name>-xml that will be created
    - port                   TCP Port that the connector uses to publish the counters.  

It will create the Kubernetes yml-file (<name>.yml) that can be used to  create the Configmap <name>-xml 

The CreateConfigMap.py scripts sets for some of the SimpliVity connector parameters default values that can be edited in the file if needed:
    
    Parameter               Default value   Comment
    - namespace             svtprometheus   adjust this to your namespace
    - timerange             30              A range in seconds (the duration from the specified point in time)
    - resolution            SECOND          The resolution (SECOND, MINUTE, HOUR, or DAY)
    - monitoringinterval    30              connector cyle time (should be >= the time to process the captured data)
    - monitor               fcn             performance data capture selector: f(ederation), c(luster), n(ode), v(irtual machine)
    - cluster                               enter a clustername if you want to limit the data capture to a single cluster
    - limit                 500             A positive integer that represents the maximum number of results to return
    - offset                -1              A positive integer that directs the service to start returning the <offset value> instance, up to the limit. Every result will be collected if the offset is set to a negative value.  
    

In [ ]:
# apply the configmap (in the following example the name svtdemo was used)
kubectl apply -f svtdemo.yml

### 2.2.2 Create the SimpliVity connector POD

In [ ]:
cat << 'EOF' | kubectl apply -f -
apiVersion: apps/v1
kind: Deployment
metadata:
  name: svtconnector
  namespace: svtprometheus      
  labels:
    app: svtconnector
spec:
  selector:
    matchLabels:
      app: svtconnector
  template:
    metadata:
      labels:
        app: svtconnector
    spec:
      containers:
        - name: svtconnector
          image: tb1378/svtconk8s
          command: ["/usr/bin/python3"]
          args: ["/opt/svt/svtpromconnector.py"]
          volumeMounts:
            - name: svtconnectorxml
              mountPath: /opt/svt/data
      volumes:
        - name: svtconnectorxml
          configMap:
            name: svtdemo-xml   # the correct name of the configmap needs to be added here. 
---
apiVersion: v1
kind: Service
metadata:
  name: svtconnector-service
  namespace: svtprometheus
spec:
  selector:
    app: svtconnector
  ports:
    - port: 9091               # The Port of that the SimpliVity connector uses
      targetPort: 9091
      protocol: TCP
EOF

##  2.3. Deploy Prometheus¶

The Prometheus POD will be deployed with service accounts, a Config Map for the input parameters and a persistent volume for the Prometheus database. We will use here three yml-files (prometheus.configmap.yml, prometheus.pv-claim.yml and prometheus.yml) instead of a single one, in order to make the process a bit easier to understand.

### 2.3.1 Create the Prometheus Configmap

The Config Map will be used to define the Prometheus monitoring jobs. Each SimpliVity connector will require a separate Prometheus monitoring job, where each job will have the following entries:

- job_name: 'simplivity-demo'
  static_configs:
  - targets: ['svtconnector:9091'] 
  honor_timestamps: true
  scrape_interval: 30s
  scrape_timeout: 10s
  metrics_path: /metrics
  scheme: htt
The above job, will scrape every 30 seconds the data from the target svtconnector:9091, where svtconnector is the name and 9091 is the port that is used by the SimpliVity Prometheus connector container (as it was deployed in 3.) The target name, port and scrape_intervall should be adjusted to the actual values of the actual implementation. 

In [ ]:
cat << 'EOF' | kubectl apply -f -
apiVersion: v1
kind: ConfigMap
metadata:
 name: prometheus-config
 namespace: svtprometheus
data:
 prometheus.yml: |
  global:
    scrape_interval:     15s # Set the scrape interval to every 15 seconds. Default is every 1 minute.
    evaluation_interval: 15s # Evaluate rules every 15 seconds. The default is every 1 minute.
  alerting:
    alertmanagers:
    - static_configs:
      - targets:
        # - alertmanager:9093

  # Load rules once and periodically evaluate them according to the global 'evaluation_interval'.
  rule_files:
    # - "first_rules.yml"
    # - "second_rules.yml"

  # A scrape configuration containing exactly one endpoint to scrape:
  # Here it's Prometheus itself.
  scrape_configs:
    # The job name is added as a label `job=<job_name>` to any timeseries scraped from this config.
    - job_name: 'prometheus'

      # metrics_path defaults to '/metrics'
      # scheme defaults to 'http'.

      static_configs:
      - targets: ['localhost:9090']
      
    - job_name: 'simplivity-demo'
      static_configs:
      - targets: ['svtdemo:9091']
      honor_timestamps: true
      scrape_interval: 30s
      scrape_timeout: 10s
      metrics_path: /metrics
      scheme: http
    - job_name: 'ctcinfrastructure'
      static_configs:
      - targets: ['svtinfra:9091']
      honor_timestamps: true
      scrape_interval: 30s
      scrape_timeout: 10s
      metrics_path: /metrics
      scheme: http

EOF

### 2.3.2 Create the persistent volume claim for the database

A persistent volume claim is used for the Prometheus database content.
The following example uses a NFS share for the persistent volume of the Prometheus database 

In [ ]:
cat << 'EOF' | kubectl apply -f -
apiVersion: v1
kind: PersistentVolume
metadata:
  name: prom-db-volume
  namespace: svtprometheus
spec:
  persistentVolumeReclaimPolicy: Recycle
  volumeMode: Filesystem
  storageClassName: "nfs"
  capacity:
    storage: 1Gi
  accessModes:
    - ReadWriteMany
  nfs:
    server: 10.1.41.12
    path: /k8s/pvc/prometheus
---
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: prometheus-pv-claim
  namespace: svtprometheus
spec:
  storageClassName: "nfs"
  accessModes:
    - ReadWriteMany
  resources:
    requests:
      storage: 1Gi
EOF

### 2.3.3 Create the Prometheus POD and Service¶
The Prometheus POD is created with the Prometheus Config Map and the persistent volume claim, that were defined in step 2.3.1 and 2.3.2. 

In [ ]:
cat << 'EOF' | kubectl apply -f -
apiVersion: v1
kind: ServiceAccount
metadata:
  name: prometheus
  namespace: svtprometheus
---
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRole
metadata:
  name: prometheus
  namespace: svtprometheus
rules:
- apiGroups: [""]
  resources:
  - nodes
  - nodes/proxy
  - services
  - endpoints
  - pods
  verbs: ["get", "list", "watch"]
- apiGroups:
  - extensions
  resources:
  - ingresses
  verbs: ["get", "list", "watch"]
- nonResourceURLs: ["/metrics"]
  verbs: ["get"]
---
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRoleBinding
metadata:
  name: prometheus
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: ClusterRole
  name: prometheus
subjects:
- kind: ServiceAccount
  name: prometheus
  namespace: svtprometheus
---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: prometheus
  namespace: svtprometheus
spec:
  replicas: 1
  selector:
    matchLabels:
      app: grafana
      tier: backend
  template:
    metadata:
      labels:
        app: grafana
        tier: backend
    spec:
      serviceAccountName: prometheus
      containers:
        - name: prometheus
          image: prom/prometheus
          args:
            - '--storage.tsdb.path=/prometheus'
            - '--storage.tsdb.no-lockfile'
            - '--config.file=/etc/prometheus/prometheus.yml'
          ports:
            - containerPort: 9090
              name: prometheus-port
          volumeMounts:
            - name: config-prometheus
              mountPath: "/etc/prometheus/"
            - name: prometheus-db
              mountPath: "/prometheus"
      volumes:     
        - name: config-prometheus
          configMap:
            name: prometheus-config
        - name: prometheus-db
          persistentVolumeClaim:
            claimName: prometheus-pv-claim
---
apiVersion: v1
kind: Service
metadata:
  name: prometheus
  namespace: svtprometheus
  labels: 
     hpecp.hpe.com/hpecp-internal-gateway: "true"
spec:
  selector:
    app: grafana
    tier: backend
  ports:
    - port: 9090
      targetPort: 9090
      protocol: TCP
  type: LoadBalancer
EOF

## 2.4 Deploy Grafana

### 2.4.1 Create the persistent volume for the Grafana Database
The following example shows a persistent volume provided by a NFS server (10.1.41.12) on the path /k8s/pvc/grafana.
The actual persistent volume the will be used, need to be adjusted to reflect your environment. 

In [ ]:
cat << 'EOF' | kubectl apply -f -
apiVersion: v1
kind: PersistentVolume
metadata:
  name: grafana-db-volume
  namespace: svtprometheus
spec:
  persistentVolumeReclaimPolicy: Retain # Recycle, Delete
  volumeMode: Filesystem
  storageClassName: "nfs"
  capacity:
    storage: 1Gi
  accessModes:
    - ReadWriteMany
  nfs:                        
    server: 10.1.41.12        
    path: /k8s/pvc/grafana
---
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: grafana-pv-claim
  namespace: svtprometheus
spec:
  storageClassName: "nfs"
  accessModes:
    - ReadWriteMany
  resources:
    requests:
      storage: 1Gi
EOF

### 2.4.2 Create the Grafana POD

In [ ]:
cat << 'EOF' | kubectl apply -f -
apiVersion: v1
kind: Service
metadata:
  name: grafana-service
  namespace: svtprometheus
spec:
  selector:
    app: grafana
    tier: frontend
  ports:
    - port: 3000
      targetPort: 3000
      protocol: TCP
  type: LoadBalancer
---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: grafana
  namespace: svtprometheus
spec:
  selector:
    matchLabels:
      app: grafana
      tier: frontend
  template:
    metadata:
      labels:
        app: grafana
        tier: frontend
    spec:     
      containers:
        - name: grafana
          image: grafana/grafana
          ports:
            - containerPort: 3000
              name: grafana-port
              
          volumeMounts:
            - name: grafana-persistent-storage
              mountPath: "/var/lib/grafana"
          env:  
            - name: GF_SMTP_ENABLED
              value: "yes"
            - name: GF_SMTP_HOST
              value: "smtp.hpe.com:25"
            - name: GF_SMTP_FROM_NAME
              value: "Beha, Thomas"
            - name: GF_SMTP_FROM_ADDRESS
              value: "thomas.beha@hpe.com"
            - name: GF_SMTP_SKIP_VERIFY
              value: "true"
            - name: GF_SERVER_HTTP_PORT
              value: "3000"
            - name: GF_INSTALL_PLUGINS
              value: "grafana-kubernetes-app"
#           - name: GF_INSTALL_PLUGINS
#             value: "ryantxu-ajax-panel"
#           - name: GF_PANELS_DISABLE_SANITIZE_HTML
#             value: "true"
#          - name: GF_AUTH_DISABLE_LOGIN_FORM
#            value: "true"
#          - name: GF_AUTH_ANONYMOUS_ENABLED
#            value: 'true'
      volumes:
        - name: grafana-persistent-storage
          persistentVolumeClaim:
            claimName: grafana-pv-claim  
EOF

# 3. Check that everything is up and running

In [ ]:
kubectl -n svtprometheus get pods,svc,configmaps,pv

# 4. Delete everything

In [ ]:
kubectl delete -f grafana.yml       
kubectl delete -f grafana.pv-claim.yml
kubectl delete -f prometheus.yml
kubectl delete -f prometheus.pv-claim.yml
kubectl delete -f prometheus.configmap.yml
kubectl delete -f svtconnector.yml 
kubectl delete -f svtdemo.yml 
kubectl delete -f namespace.yml 